## Deliverable 3. Create a Travel Itinerary Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import maps_api_key

# Configure gmaps
gmaps.configure(api_key=maps_api_key)

In [20]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_data_df = pd.read_csv("WeatherPy_vacation.csv")
vacation_data_df


,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,2,Kavieng,PG,81.09,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
1,3,Kapaa,US,76.12,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,4,Port Elizabeth,ZA,70.09,clear sky,-33.9180,25.5701,39 On Nile Guest House
3,7,Rikitea,PF,78.46,light rain,-23.1203,-134.9692,People ThankYou
4,8,Hobart,AU,62.02,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
...,...,...,...,...,...,...,...,...
317,662,Auki,SB,77.09,light rain,-8.7676,160.7034,Auki Motel
318,664,Mount Gambier,AU,60.71,overcast clouds,-37.8333,140.7667,The Commodore
319,665,San Carlos De Bariloche,AR,85.75,clear sky,-41.1456,-71.3082,Hotel Patagonia
320,667,Kirakira,SB,79.84,light rain,-10.4544,161.9205,SanBiz Lodge


In [21]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_data_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_data_df[["Lat", "Lng"]]

In [24]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_data_df[["Lat", "Lng"]]
max_temp = vacation_data_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [28]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_data_df.loc[vacation_data_df["City"] =="Laguna" ]
vacation_end = vacation_data_df.loc[vacation_data_df["City"] =="Laguna" ]
vacation_stop1 = vacation_data_df.loc[vacation_data_df["City"] =="Healdsburg" ]
vacation_stop2 = vacation_data_df.loc[vacation_data_df["City"] =="Half Moon Bay" ]
vacation_stop3 = vacation_data_df.loc[vacation_data_df["City"] =="Pacific Grove" ]

In [29]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}, stop 1: {stop1},stop 2: {stop2},stop 3: {stop3}')

Start: (38.421, -121.4238), End: (38.421, -121.4238), stop 1: (38.6105, -122.8692),stop 2: (37.4636, -122.4286),stop 3: (36.6177, -121.9166)


In [30]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
fig = gmaps.figure()
vacation_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(vacation_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [32]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_end,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,256,Laguna,US,80.44,few clouds,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
1,256,Laguna,US,80.44,few clouds,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
2,88,Healdsburg,US,80.10,clear sky,38.6105,-122.8692,Best Western Dry Creek Inn
3,264,Half Moon Bay,US,79.61,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
4,331,Pacific Grove,US,84.61,clear sky,36.6177,-121.9166,Lovers Point Inn


In [34]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))